In [3]:
import numpy as np

In [101]:
def fit(x_train,y_train):
    result={}
    class_values=set(y_train)
    for current_class in class_values:
        result[current_class]={}
        result["total_data"]=len(y_train)
        current_class_row=(y_train==current_class)
        x_train_current=x_train[current_class_row]
        y_train_current=y_train[current_class_row]
        num_features=x_train.shape[1]
        result[current_class]["total_count"]=len(y_train_current)
        for j in range(1,num_features+1):
            result[current_class][j]={}
            all_possible_values=set(x_train[:,j-1])
            for current_values in all_possible_values:
                result[current_class][j][current_values]=(x_train_current[:,j-1]==current_values).sum()
    return result            

In [114]:
def probablity(dictonary,x,current_class):
    output=np.log(dictonary[current_class]["total_count"])-np.log(dictonary["total_data"])
    num_fetures=len(dictonary[current_class].keys())-1
    for j in  range(1,num_fetures+1):
        xj=x[j-1]
        count_current_class_with_value_xj=dictonary[current_class][j][xj]+1
        count_current_class=dictonary[current_class]["total_count"]+len(dictonary[current_class][j].keys())
        current_xj_probablity=np.log(count_current_class_with_value_xj)-np.log(count_current_class)
        output=output+current_xj_probablity
    return output      

In [103]:
def predictsinglepoint(diconary,x):
    classes=dictonary.keys()
    best_p=-1000
    best_class=-1
    first_run=True
    for current_classes in classes:
        if (current_classes=="total_data"):
            continue
        p_current_class=probablity(dictonary,x,current_classes)
        if(first_run or  p_current_class>best_p):
            best_p=p_current_class
            best_class=current_classes
        first_run=False
    return best_class    

In [100]:
def predict(dictonary,x_test):
    y_pred=[]
    for x in x_test:
        x_class=predictsinglepoint(dictonary,x)
        y_pred.append(x_class)
    return y_pred    

In [104]:
def makeLabelled(column):
    second_limit=column.mean()
    first_limit=.5*second_limit
    third_limit=1.5*second_limit
    
    for i in range(0,len(column)):
        if(column[i]<first_limit):
            column[i]=0;
        elif (column[i]<second_limit):
            column[i]=1
        elif (column[i]<third_limit):
            column[i]=2;
        else:
            column[i]=3
    return column

In [105]:
from sklearn import datasets as db
iris=db.load_iris()

In [106]:
x=iris.data
y=iris.target

In [107]:
for i in range(0,x.shape[1]):
    x[:,i]=makeLabelled(x[:,i])

In [108]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0,test_size=.25)

In [109]:
dictonary=fit(x_train,y_train)

In [117]:
y_pred=predict(dictonary,x_test)

In [120]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        13
          1       0.94      1.00      0.97        16
          2       1.00      0.89      0.94         9

avg / total       0.98      0.97      0.97        38

[[13  0  0]
 [ 0 16  0]
 [ 0  1  8]]


In [123]:
from sklearn.naive_bayes import GaussianNB
clf=GaussianNB()
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      0.85      0.92        13
          1       0.76      1.00      0.86        16
          2       1.00      0.67      0.80         9

avg / total       0.90      0.87      0.87        38

[[11  2  0]
 [ 0 16  0]
 [ 0  3  6]]


In [128]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      0.85      0.92        13
          1       0.00      0.00      0.00        16
          2       0.36      1.00      0.53         9

avg / total       0.43      0.53      0.44        38

[[11  2  0]
 [ 0  0 16]
 [ 0  0  9]]
